# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [1]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [2]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [3]:
%store -r numerosKalaba typeKalaba
numerosKalaba=[1,2,3,4,5]
numerosKalaba=[18]
print numerosKalaba
typeKalaba="Kanonik"
%store numerosKalaba 
%store typeKalaba

[18]
Stored 'numerosKalaba' (list)
Stored 'typeKalaba' (str)


In [4]:
home = expanduser("~")

if typeKalaba!="Kanonik":
    repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
    serie=repertoire+"22-"
    nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]
else:
    repertoire=home+"/Library/Mobile Documents/com~apple~CloudDocs/Downloads/"
    serie=repertoire+"23-Kanoniks/"
    nomsKalabas=[serie+"Kanonik-%02d-ok/"%num for num in numerosKalaba]

In [5]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [6]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    
    lexiqueTestPrep=getExemple(stems["PREP"]).lower()
    lexiqueTestNoun=getExemple(stems["NOM"]).lower()
    lexiqueTestHyper=getExemple(stems["ADJ"]).lower()
    
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    # classesNom=PFM.lexique.formeLexeme[mot.lower()][0].split('.')[1:]
    # [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/Library/Mobile Documents/com~apple~CloudDocs/Downloads/23-Kanoniks/Kanonik-18-ok/
NOM : Genre Nombre
VER : Temps Pers Genre
ADJ : Genre Nombre
PRO : Genre Nombre
DET : Genre Nombre
head stems
head stems,NOM


### Bilan

In [7]:
stems["NOM"]

{'Ani': {'belu': ['mammouth', 'mammouths'],
  'den': ['tortue'],
  'imat': ['poisson', 'poissons'],
  'liru': ['licorne'],
  'rita': ['oiseau', 'oiseaux'],
  'tud': ['lapin']},
 'Hum': {'Subo': ['chevalier', 'chevaliers'],
  'almel': ['Almael'],
  'anok': ['Anouk'],
  'god': ['fille', 'filles'],
  'letun': ['magicien'],
  'lom': [u'L\xe9ina'],
  'men': ['enfant', 'enfants']},
 'Ina': {'Nis': ['vent'],
  'Sogi': ['tartine', 'tartines'],
  'Zepu': ['pomme', 'pommes'],
  'Zili': ['maison'],
  'bod': ['grotte'],
  'bul': ['cri', 'cris'],
  'dero': [u'cr\xe2ne', u'cr\xe2nes'],
  'dodr': ['orage'],
  'finov': ['Afior'],
  'fisn': ['os', 'oS'],
  'fite': ['aile', 'ailes'],
  'gef': [u'oxyg\xe8ne'],
  'gone': ['lueur'],
  'kar': ['ciel'],
  'kubl': ['stylo'],
  'log': ['rambarde'],
  'lole': [u'obscurit\xe9'],
  'maf': [u'rivi\xe8re'],
  'mig': ['caillou', 'cailloux'],
  'muvin': ['soleil'],
  'nak': ['baguette'],
  'put': ['pelle'],
  'rasa': ['lunette', 'lunettes'],
  'rev': ['escalier'],
  

In [8]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
# print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

blanc
sur


In [9]:
paradigmes.getSigmas(["VER"])

[u'VER, Temps=PRS, Pers=3Sg, Genre=Ani',
 u'VER, Temps=PRS, Pers=3Sg, Genre=Ina',
 u'VER, Temps=PRS, Pers=3Sg, Genre=Hum',
 u'VER, Temps=PRS, Pers=3Du, Genre=Ani',
 u'VER, Temps=PRS, Pers=3Du, Genre=Ina',
 u'VER, Temps=PRS, Pers=3Du, Genre=Hum',
 u'VER, Temps=PRS, Pers=3Pl, Genre=Ani',
 u'VER, Temps=PRS, Pers=3Pl, Genre=Ina',
 u'VER, Temps=PRS, Pers=3Pl, Genre=Hum',
 u'VER, Temps=FUT, Pers=3Sg, Genre=Ani',
 u'VER, Temps=FUT, Pers=3Sg, Genre=Ina',
 u'VER, Temps=FUT, Pers=3Sg, Genre=Hum',
 u'VER, Temps=FUT, Pers=3Du, Genre=Ani',
 u'VER, Temps=FUT, Pers=3Du, Genre=Ina',
 u'VER, Temps=FUT, Pers=3Du, Genre=Hum',
 u'VER, Temps=FUT, Pers=3Pl, Genre=Ani',
 u'VER, Temps=FUT, Pers=3Pl, Genre=Ina',
 u'VER, Temps=FUT, Pers=3Pl, Genre=Hum']

In [10]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [11]:
regles.getRules("VER",'CF=V1, Temps=PRS, Pers=3PL')
# regles.getRules("PRO","Nombre=Du, Genre=C")

[]

In [12]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{}

In [13]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

stylo.Ina kubl
Anouk.Hum anok
PRO kad
planète.Ina suN
poisson.Ani imat
tartine.Ina Sogi
baguette.Ina nak
pomme.Ina Zepu
oiseau.Ani rita
jouer vub
être pofl
DEM r
enfant.Hum men
aile.Ina fite
Afior.Ina finov
aider duta
découvrir mon
DEF f
blanc tof
étoile.Ina ril
transformer givu
oxygène.Ina gef
noir neg
caillou.Ina mig
devenir bud
lasagne.Ina ruvo
fille.Hum god
manger fesna
avec tuk
pousser tave
grotte.Ina bod
long zlu
en pod
casser kagu
magicien.Hum letun
os.Ina fisn
sur daf
atterrir vekoz
vent.Ina Nis
briller getek
Almael.Hum almel
obscurité.Ina lole
chevalier.Hum Subo
escalier.Ina rev
gros Nud
feu.Ina sega
vert lidmu
grand moke
feuille.Ina tuv
maison.Ina Zili
rambarde.Ina log
voir ekat
devant Nis
lunette.Ina rasa
sombre dev
orage.Ina dodr
cri.Ina bul
mammouth.Ani belu
crâne.Ina dero
de plok
lapin.Ani tud
Léina.Hum lom
prendre kunoso
sous lap
rivière.Ina maf
longer teke
capturer nap
licorne.Ani liru
regarder ponb
lave.Ina sak
ciel.Ina kar
à gum
tortue.Ani den
magique poS
pour vri
écr

In [14]:
ding()